# Data Prep

This project uses data from two sources. The first is the [Top Streamers on Twitch](https://www.kaggle.com/datasets/aayushmishra1512/twitchdata/data) dataset from Kaggle, with information on the top 1000 Twitch streamers in 2020.

The second data source comes from the [Twitch Earnings Leaderboard](https://twitch.pages.dev), showing the earnings of top twitch streamers from August 2019 until October 2021. 


## Top Twitch Stremers Data

First download the twitch streamers data and clean column names.

In [1]:
import pandas as pd

In [2]:
# Read data and set column names to snake_case
df = pd.read_csv("data/twitch-top-streamers.csv")
df.columns = [col.lower().replace(' ', '_').replace('(', '_').replace(')', '') for col in df.columns]

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   channel              1000 non-null   object
 1   watch_time_minutes   1000 non-null   int64 
 2   stream_time_minutes  1000 non-null   int64 
 3   peak_viewers         1000 non-null   int64 
 4   average_viewers      1000 non-null   int64 
 5   followers            1000 non-null   int64 
 6   followers_gained     1000 non-null   int64 
 7   views_gained         1000 non-null   int64 
 8   partnered            1000 non-null   bool  
 9   mature               1000 non-null   bool  
 10  language             1000 non-null   object
dtypes: bool(2), int64(7), object(2)
memory usage: 72.4+ KB


In [4]:
df

,channel,watch_time_minutes,stream_time_minutes,peak_viewers,average_viewers,followers,followers_gained,views_gained,partnered,mature,language
0,xQcOW,6196161750,215250,222720,27716,3246298,1734810,93036735,True,False,English
1,summit1g,6091677300,211845,310998,25610,5310163,1370184,89705964,True,False,English
2,Gaules,5644590915,515280,387315,10976,1767635,1023779,102611607,True,True,Portuguese
3,ESL_CSGO,3970318140,517740,300575,7714,3944850,703986,106546942,True,False,English
4,Tfue,3671000070,123660,285644,29602,8938903,2068424,78998587,True,False,English
...,...,...,...,...,...,...,...,...,...,...,...
995,LITkillah,122524635,13560,21359,9104,601927,562691,2162107,True,False,Spanish
996,빅헤드 (bighead033),122523705,153000,3940,793,213212,52289,4399897,True,False,Korean
997,마스카 (newmasca),122452320,217410,6431,567,109068,-4942,3417970,True,False,Korean
998,AndyMilonakis,122311065,104745,10543,1153,547446,109111,3926918,True,False,English


## Twitch Earnings Data

There is no clear way to download the earnings data, so we will scrape it using the `bs4` library.

In [5]:
import scrape_twitch_earnings as ste

# this function will scrape the twitch earnings data and save it to a csv file in the data directory
# takes around 4 minutes to run
# ste.scrape_twitch_earnings(max_clicks = 100, file_path = 'data/twitch-earnings.csv')

In [6]:
# read scraped earnings data
earnings_df = pd.read_csv("data/twitch-earnings.csv")
earnings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ranking       10000 non-null  int64  
 1   name          10000 non-null  object 
 2   earnings_usd  10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [7]:
earnings_df

,ranking,name,earnings_usd
0,1,CriticalRole,9626712.16
1,2,xQcOW,8454427.17
2,3,summit1g,5847541.17
3,4,Tfue,5295582.44
4,5,NICKMERCS,5096642.12
...,...,...,...
9995,9996,Quig,23528.80
9996,9997,BoozyCruzy,23527.84
9997,9998,Cewpins,23526.21
9998,9999,Megasnoop,23520.87


Combine twitch streamers data with earnings data.

In [8]:
df_all = df.merge(earnings_df, left_on='channel', right_on='name', how='left')
df_all = df_all.drop(columns=['name'])

Check for missing values

In [9]:
df_all.isnull().sum()

channel                  0
watch_time_minutes       0
stream_time_minutes      0
peak_viewers             0
average_viewers          0
followers                0
followers_gained         0
views_gained             0
partnered                0
mature                   0
language                 0
ranking                212
earnings_usd           212
dtype: int64

Exclude rows without earnings.

In [10]:
df_all = df_all[df_all['earnings_usd'].notnull()]

In [11]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 788 entries, 0 to 999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   channel              788 non-null    object 
 1   watch_time_minutes   788 non-null    int64  
 2   stream_time_minutes  788 non-null    int64  
 3   peak_viewers         788 non-null    int64  
 4   average_viewers      788 non-null    int64  
 5   followers            788 non-null    int64  
 6   followers_gained     788 non-null    int64  
 7   views_gained         788 non-null    int64  
 8   partnered            788 non-null    bool   
 9   mature               788 non-null    bool   
 10  language             788 non-null    object 
 11  ranking              788 non-null    float64
 12  earnings_usd         788 non-null    float64
dtypes: bool(2), float64(2), int64(7), object(2)
memory usage: 75.4+ KB


In [12]:
df_all

,channel,watch_time_minutes,stream_time_minutes,peak_viewers,average_viewers,followers,followers_gained,views_gained,partnered,mature,language,ranking,earnings_usd
0,xQcOW,6196161750,215250,222720,27716,3246298,1734810,93036735,True,False,English,2.0,8454427.17
1,summit1g,6091677300,211845,310998,25610,5310163,1370184,89705964,True,False,English,3.0,5847541.17
2,Gaules,5644590915,515280,387315,10976,1767635,1023779,102611607,True,True,Portuguese,12.0,2844985.18
3,ESL_CSGO,3970318140,517740,300575,7714,3944850,703986,106546942,True,False,English,29.0,1903580.27
4,Tfue,3671000070,123660,285644,29602,8938903,2068424,78998587,True,False,English,4.0,5295582.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,ZiggyDLive,123398085,99075,18437,1227,173807,15899,4249580,True,False,English,1617.0,124020.20
993,LAGTVMaximusBlack,123125340,161535,7138,725,256353,13251,2310313,True,True,English,401.0,380499.30
995,LITkillah,122524635,13560,21359,9104,601927,562691,2162107,True,False,Spanish,4318.0,51515.60
998,AndyMilonakis,122311065,104745,10543,1153,547446,109111,3926918,True,False,English,586.0,283763.15


# Save

In [13]:
df_all.to_csv("data/twitch-data-all.csv", index=False)